# Acordo de chaves para comunicação confidencial e autenticada

Construção de uma sessão síncrona de comunicação segura entre dois agentes (o Emitter e o Receiver), combinando os seguintes elementos constituintes:
a cifra simétrica TAES (AES na sua versão tweak) usando autenticação do criptograma em cada superbloco; 
o protocolo de acordo de chaves Diffie-Hellman com verificação da chave, e autenticação dos agentes através do esquema de assinaturas DSA.
Use chaves DH/DSA de 3072 bits para acordo de chaves e autenticação dos agentes. Tenha em muita atenção os tempos de vida diferentes para as chaves acordadas (chaves de sessão) e as chaves no esquema de assinaturas (chaves “permanentes”).

## Connection

In [8]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        self.lproc = Process(target = left, args=(left_end,))
        self.rproc = Process(target = right, args=(right_end,))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

## Primitivas criptográficas

O uso do esquema Encrypt-then-MAC permite garantir a autenticidade, integridade e confidencialidade das mensagens trocadas entre o `Emitter` e o `Receiver`.
Como tal, a cifra simétrica escolhida foi o AES (Advanced Encryption Standard) em modo CFB (Cipher Feedback Mode).
O MAC (Message Authentication Code) escolhido foi o HMAC (Hash-MAC) com SHA-256 que permite, por sua vez, garantir a integridade e autenticidade da mensagem trocada.

In [14]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.ciphers import algorithms, modes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import os

# DH Parameters
P = 99494096650139337106186933977618513974146274831566768179581759037259788798151499814653951492724365471316253651463342255785311748602922458795201382445323499931625451272600173180136123245441204133515800495917242011863558721723303661523372572477211620144038809673692512025566673746993593384600667047373692203583
G = 44157404837960328768872680677686802650999163226766694797650810379076416463147265401084491113667624054557335394761604876882446924929840681990106974314935015501571333024773172440352475358750668213444607353872754650805031912866692119819377041901642732455911509867728218394542745330014071040326856846990119719675

tweak = os.urandom(16)

def verifyHMAC(key, tag, ctxt):
    """ Computes the HMAC of a given cipher text and checkes whether it matches with
        the given tag i.e. validates the authenticity and integrity of the message
    """
    try:
        h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
        h.update(ctxt)
        h.verify(tag)                        #exception on MAC failure
        return True
    except:
        return False

def extractMessage(key, iv, tweak_h, ctxt):
    """ Extracts the plain text from a cryptogram resulting from a tweakable cipher
        where ptxt = dec(ctxt ^ hash(tweak)) ^ hash(tweak)
    """
    decryptor = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend()).decryptor()
    ptxt = (decryptor.update(ctxt ^ tweak_h) + decryptor.finalize()) ^ tweak_h
    return ptxt

def deriveKey(key_base, key_size, info):
    """ Expands key from key_base to key_size bytes
    """
    key = HKDF(hashes.SHA256(), length=key_size, salt=None, info=info, backend=default_backend()).deriveKey(key_base)
    return key
    

## Emitter

In [15]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import (Cipher, algorithms, modes)
import os

def Emitter(conn):
    pwd = getpass('Emitter password: ')
    try:
        parameters = dh.DHParameterNumbers(P,G).parameters(backend=default_backend()) #receive DH parameters
        priv_key = parameters.generate_private_key()
        conn.send(priv_key.public_key())                  #send g^y mod p
        g_x = conn.recv()
        s_key = priv_key.exchange(g_x)  #generate shared secret
        
        #TODO: derive h_key and c_key from s_key
        
        header = 'msgheader'.encode("utf-8")
        content = 'Here is a sample message for testing'
        
        iv = os.urandom(16)
        cipher = Cipher(algorithms.AES(c_key),modes.CFB(iv),backend).encryptor()
        
        #TODO: compute tweak hash
        content_new = cipher.update(content.encode("utf-8")) + cipher.finalize()
        # tweak_h = hash of tweak value
        #tweak_h = hmac.HMAC(h_key, hashes.SHA256(), backend=default_backend()).update(tweak)
        
        #TODO: compute HMAC of header+content_new+iv+tweak_h and send in tag
        m_hmac = hmac.HMAC(h_key, hashes.SHA256(), backend=default_backend()).update(content_new)
        conn.send(tag)
        
        payload = {'header': header, 'content': content_new, 'iv':iv, 'm_hmac': m_hmac, 'tweak_h': tweak_h}
        conn.send(payload)
        conn.close()
    except:
       print("Emitter: Error")

## Receiver

O Receiver, após a troca de chaves, e a cada mensagem recebida, efetua uma verificação da autenticidade e integridade da mesma com recurso à função `verifyHMAC` que recebe como parâmetro a chave usada no cálculo do MAC, o MAC esperado e o *input* sobre o qual este deve ser calculado.

Caso a mensagem recebida não tenha sido modificada, o `Receiver` procede ao processo de decifragem. Dado tratar-se de
uma mensagem cifrada com **TAES** é necessário manipular o criptograma de modo a obter a mensagem original, sendo útil ter em conta a seguinte expressão para o processo de decifragem:

$$ ctxt = E_k(ptxt \oplus h(tweak)) \oplus h(tweak)$$

onde:
- $ctxt$ : criptograma recebido
- $ptxt$ : mensagem original
- $tweak$: tweak/nonce

como tal, e tendo ainda em conta que $ D_k = E^{-1}_k $ , a obtenção do texto limpo original($ptxt$) pode ser conseguida da seguinte maneira:

$$ ptxt = D_k(ctxt \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak)) \oplus h(tweak) \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak))) \oplus h(tweak) \rightarrow ptxt \oplus h(tweak) \oplus h(tweak) \rightarrow ptxt $$

Este processo é implementado pela função `extractMessage` que recebe como parâmetros a chave acordada (`key`), o vetor de inicialização (`iv`), o *hash* do *tweak* (`tweak_h`) e o criptograma (`ctxt`), devolvendo o resultado de aplicar a expressão anteriormente apresentada.


In [16]:
def Receiver(conn):
    pwd = getpass('Reciever password: ')
    try:
        parameters = dh.DHParameterNumbers(P,G).parameters(backend=default_backend())
        g_y = conn.recv()
        priv_key = parameters.generate_private_key()
        conn.send(priv_key.public_key())                   #send g^x mod y
        
        s_key = priv_key.exchange(g_y)
        d_key = deriveKey(key, 32, b"DH Key Exchange")
        c_key = d_key[:128]
        h_key = d_key[128:]
        
        payload = conn.recv()
        header = payload['header']
        ctxt = payload['content']
        iv = payload['iv']
        m_hmac = payload['m_hmac']
        tweak_h = payload['tweak_h']
        
        if verifyHMAC(h_key, m_hmac, header+ctxt+iv+tweak_h):
            msg = extractMessage(c_key, ctxt, tweak_h)
            print("Received: " + msg.decode("utf-8"))
        conn.close()
    except:
        print("Receiver: Erro na decifragem")

O estabelecimento de uma conexão entre os dois intervenientes (`Emitter` e `Receiver`)  é feita invocando o método `auto` da classe `Connection` que cria uma comunicação síncrona entre ambos. 
Este sincronismo é exigido pelo protocolo de troca de chaves (Diffie Hellman)